# <center><b>Reinforcement Learning for Quadruped Locomotion</b></center>

<div align="center">

<h3>KAIST DRCD Lab</h3>


<b>Instructor</b><br>
<a href="https://dynamicrobot.kaist.ac.kr/people.html">Hae-Won Park</a>
(haewonpark@kaist.ac.kr)

<br>

<b>Teaching Assistants</b><br>
<a href="https://kdyun0118.github.io/">Dongyun Kang</a>
(kdong7309@kaist.ac.kr)<br>
<a href="https://github.com/parkjahun42">Jaehyun Park</a>
(parkjahun42@kaist.ac.kr)<br>
<a href="https://github.com/sowoolee">Sowoo Lee</a>
(dlthdn@kaist.ac.kr)


<img src="https://drive.google.com/uc?id=1CGEKK43-VXknF_8t51pec6wp01szzVNX"
     width="900">




</div>

<br>
<br>

## Abstract

이 튜토리얼은 **MuJoCo 시뮬레이터 환경에서 강화학습을 통해 사족로봇(Unitree Go1)의 보행제어기를 학습**해보는 것을 목표로 합니다.
이를 위해서 PPO(Proximal Policy Optimization)를 기반으로 한 보행 정책 학습 과정을 단계별로 설명하며,

- Go1 MuJoCo 환경 구성 및 관측/행동 설계
- PPO 학습 파이프라인과 주요 하이퍼파라미터
- 사전 학습된 정책(pretrained policy)을 활용한 파인튜닝
- Reward ablation을 통한 보행 성능 비교 분석

등의 내용을 포함합니다.

<br>


References

<small>
[1] https://github.com/nimazareian/quadruped-rl-locomotion <br>
[2] Todorov, Emanuel, Tom Erez, and Yuval Tassa. "Mujoco: A physics engine for model-based control." 2012 IEEE/RSJ.
</small>


---

## 0. Environment Setup

본 튜토리얼은 **Google Colab** 환경에서 실행할 수 있도록 작성되었습니다.

아래 순서에 따라 실행 환경을 준비해 주세요.

1. **런타임 유형을 CPU로 설정**하고 런타임에 연결합니다.
<div align="center">
  <img src="https://drive.google.com/uc?id=1tamjiJCNrQ5W-3KFr6JAoILyXDLMHoPQ"
       width="300">
</div>

2. 제공된 GitHub 레포지토리( https://github.com/DrcdKAIST/RL_DEMO.git )를 **clone**하여 코드 베이스를 준비합니다.
3. MuJoCo 및 강화학습 실험에 필요한 **의존성 패키지들을 설치**합니다.


In [9]:
# Clone repository
import os, sys

import yaml

os.chdir("/content")
if not os.path.isdir("RL_DEMO"):
  !git clone https://github.com/DrcdKAIST/RL_DEMO.git
else:
  print("Cloned Directory already exists")

os.chdir("/content/RL_DEMO")
print("Current Directory: ", os.getcwd())

sys.path.insert(0, "/content/RL_DEMO")
os.environ["MUJOCO_GL"] = "egl"

Cloned Directory already exists
Current Directory:  /content/RL_DEMO


In [10]:
# Install dependencies
!pip install torch numpy tensorboard gymnasium==0.29.1 stable-baselines3==2.3.0 mujoco==3.1.5 imageio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
Reason for being yanked: Loading broken with PyTorch 1.13
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.5/243.5 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.2.3
    Uninstalling gymnasium-1.2.3:
      Successfully uninstalled gymnasium-1.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.1 which is incompatible.


---

<br>

## 1. Go1 MuJoCo Environment

본 섹션에서는 **MuJoCo 기반의 Go1 로봇 보행환경**에 대하여 설명합니다.

학습 환경은 `Go1MujocoEnv` 클래스에 구현되어 있으며, 로봇이 정해진 범위 내의 직진 command를 안정적으로 추종하는 문제를 **마르코프 결정 과정(Markov Decision Process, MDP)** 의 형태로 구성합니다.

<br>

### 1.1 Simulation Setup

- **물리 시뮬레이터**: MuJoCo
- **로봇 모델**: Unitree Go1
- **시뮬레이션 정의**:
  시뮬레이션 환경은 `unitree_go1/scene_position.xml` 파일에 정의되어 있으며,
  로봇 모델, 지면, 조명, 카메라 설정을 포함합니다.
- **로봇 설정**:
  관절 액추에이터, PD 제어 이득, 기구학 구조는 `go1_position.xml` 파일에 정의되어 있습니다.

<br>



In [11]:
# investigate scene xml files
!sed -n '1,200p' unitree_go1/scene_position.xml

<mujoco model="go1 scene">
  <include file="go1_position.xml"/>

  <statistic center="0 0 0.1" extent="0.8"/>

  <visual>
    <headlight diffuse="0.6 0.6 0.6" ambient="0.3 0.3 0.3" specular="0 0 0"/>
    <rgba haze="0.15 0.25 0.35 1"/>
    <global azimuth="120" elevation="-20"/>
  </visual>

  <asset>
    <texture type="skybox" builtin="gradient" rgb1="0.3 0.5 0.7" rgb2="0 0 0" width="512" height="3072"/>
    <texture type="2d" name="groundplane" builtin="checker" mark="edge" rgb1="0.2 0.3 0.4" rgb2="0.1 0.2 0.3"
      markrgb="0.8 0.8 0.8" width="300" height="300"/>
    <material name="groundplane" texture="groundplane" texuniform="true" texrepeat="5 5" reflectance="0.2"/>
  </asset>

  <worldbody>
    <light pos="0 0 1.5" dir="0 0 -1" directional="true"/>
    <geom name="floor" size="0 0 0.05" type="plane" material="groundplane"/>
  </worldbody>
</mujoco>


In [12]:
# investigate robot xml files
!sed -n '1,1000p' unitree_go1/go1_position.xml

<mujoco model="go1">
  <compiler angle="radian" meshdir="assets" autolimits="true" inertiafromgeom="false"/>

  <option cone="elliptic" impratio="100"/>

  <default>
    <default class="go1">
      <!-- <geom friction="0.6" margin="0.001" condim="1"/> -->
      <joint axis="0 1 0" damping="1.0" armature="0.01" frictionloss="0.1"/>
      <position kp="20" kv="1" forcerange="-23.7 23.7"/>
      <default class="abduction">
        <joint axis="1 0 0" range="-0.863 0.863"/>
        <position ctrlrange="-0.863 0.863"/>
      </default>
      <default class="hip">
        <joint range="-0.686 4.501"/>
        <position ctrlrange="-0.686 4.501"/>
      </default>
      <default class="knee">
        <joint range="-2.818 -0.888"/>
        <position forcerange="-35.55 35.55" ctrlrange="-2.818 -0.888"/>
      </default>
      <default class="visual">
        <geom type="mesh" contype="0" conaffinity="0" group="2" material="dark"/>
      </default>
      <default class="collision">
        <geom 



### 1.2 Environment Configuration

강화학습 환경은 다음과 같이 구현되어 있습니다.

- `Go1MujocoEnv` 클래스 (`src/go1_mujoco_env.py` 참고)
  - 환경의 동역학 및 MDP 구성 로직
  - gymnasium의 MujocoEnv 클래스 상속
- `src/envs.yaml`은 환경 설정 파일로, 다음 항목들을 포함합니다.
  - 에피소드 길이
  - Reward 가중치
  - 목표 속도(command) 범위
  - Observation 스케일링
  - 조기 종료(early termination) 조건
  등등

<br>



### 1.3 Environment Step Loop
(`src/go1_mujoco_env.py` 의  `step` method 참고)

<br>

각 타임스텝마다 다음과 같은 절차가 수행됩니다.

1. 정책(policy)이 현재 상태를 기반으로 행동(action)을 출력합니다.
2. 시뮬레이터가 고정된 프레임 수(`frame_skip`)만큼 진행됩니다.
3. Observation, reward, 종료 조건이 계산됩니다.
4. 해당 전이(transition)가 학습을 위해 저장됩니다.

<br>

### 1.4 Observation Space
(`src/go1_mujoco_env.py` 의  `_get_obs` method 참고)

<br>

Observation 벡터는 다음과 같은 정보를 포함합니다.

- 로봇 베이스의 선형 및 각속도
- 중력 방향이 투영된 벡터
- 목표 이동 속도(command)
- 관절 위치 및 속도
- 이전 타임스텝의 action (제어 입력의 연속성 확보 목적)

모든 observation은 학습 안정성을 위해 사전에 정의된 범위로 스케일링 및 클리핑됩니다.

<br>

### 1.5 Reward and Termination
(`src/go1_mujoco_env.py` 의  `_get_reward` method 참고)

<br>


Reward 함수와 종료 조건은 다음 모듈에 분리되어 구현되어 있습니다.

- `src/mdp/reward.py`
- `src/mdp/termination.py`

이와 같은 모듈화된 구조를 통해 개별 reward 항을 손쉽게 수정하거나 제거할 수 있습니다.

In [13]:
# Check observation / action space

import importlib
import numpy as np
import src.go1_mujoco_env as go1_env

importlib.reload(go1_env)

# Create environment (no rendering)
env = go1_env.Go1MujocoEnv(
    prj_path="/content/RL_DEMO",
    render_mode=None,
)

obs, info = env.reset()

print(f"Observation shape: {np.array(obs).shape}\n")
print(f"Action space: {env.action_space}\n")
print(f"Observation space: {env.observation_space}\n")

# Take one random step
action = env.action_space.sample()
obs, reward, terminated, truncated, info = env.step(action)

print("One-step reward:", reward)
print("Terminated:", terminated)

env.close()

Observation shape: (48,)

Action space: Box([-0.863     -1.3859999 -1.218     -0.863     -1.3859999 -1.218
 -0.863     -1.586     -1.218     -0.863     -1.586     -1.218    ], [0.863     3.8009999 0.712     0.863     3.8009999 0.712     0.863
 3.6009998 0.712     0.863     3.6009998 0.712    ], (12,), float32)

Observation space: Box(-inf, inf, (48,), float32)

One-step reward: -2.394462011389863
Terminated: False


In [14]:
import numpy as np
import importlib

import os
import gc
import time
import imageio
import torch

from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback, CallbackList
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
from IPython.display import Video, display
from pathlib import Path

import src.go1_mujoco_env as go1_env

from src.utils.reward_logging_callback import RewardLoggingCallback

DEFAULT_CAMERA_CONFIG = {
    "azimuth": 90.0,
    "distance": 3.0,
    "elevation": -25.0,
    "lookat": np.array([0., 0., 0.]),
    "fixedcamid": 0,
    "trackbodyid": -1,
    "type": 2,
}

policy_cfg_path = Path("/content/RL_DEMO/src/params.yaml")
with policy_cfg_path.open("r", encoding="utf-8") as f:
    policy_cfg = yaml.safe_load(f)

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [15]:
# colab에서 실행하기 위한 설정
policy_cfg['n_envs'] = 12
policy_cfg['policy']['batch_size'] = 64

print(policy_cfg['n_envs'])
print(policy_cfg['policy']['batch_size'])

12
64


---

<br>

## 2. PPO Code Review

### 2.1 Stable-Baselines3

본 튜토리얼에서는 **Stable-Baselines3(SB3)** 라이브러리에 구현된
**PPO(Proximal Policy Optimization)** 알고리즘을 사용하였습니다.

SB3는 PyTorch 기반의 라이브러리로,
PPO, SAC, TD3 등의 다양한 강화학습 알고리즘을 제공합니다.

PPO는 actor-critic 구조를 기반으로 하며,
정책 업데이트 시 변화 폭을 제한(clipping)하여
학습 안정성을 확보하는 것이 특징입니다.

이 섹션에서는 수식적 유도보다는,
SB3의 PPO 모듈이 이번 보행제어기의 학습과정에서 어떻게 활용되었는지에 초점을 맞춥니다.



In [16]:
import stable_baselines3
import stable_baselines3.ppo.ppo as ppo_module
import inspect, os

print("stable-baselines3 version:", stable_baselines3.__version__)
print("PPO module path:", os.path.abspath(ppo_module.__file__))
print("PPO class:", ppo_module.PPO)

stable-baselines3 version: 2.3.0
PPO module path: /usr/local/lib/python3.12/dist-packages/stable_baselines3/ppo/ppo.py
PPO class: <class 'stable_baselines3.ppo.ppo.PPO'>


<br>

### 2.2 Vectorized Environments

PPO는 on-policy 알고리즘이므로,
매 업데이트마다 많은 샘플을 필요로 합니다.

이를 위해 SB3는 여러 개의 환경을 동시에 실행하는
**vectorized environment**를 지원합니다.

본 튜토리얼에서는 `SubprocVecEnv`를 사용하여
여러 개의 Go1 환경을 병렬로 실행합니다.
이는 MuJoCo 기반 시뮬레이션에서 샘플 수집 속도를
크게 향상시켜 줍니다.

<br>

### 2.3 PPO 에이전트 생성

이제 병렬 환경을 기반으로 PPO 에이전트를 생성합니다.

본 튜토리얼에서는 두 가지 학습 방식을 지원합니다.

- 사전 학습된 policy를 불러와 추가 학습(finetuning)
- 네트워크를 새로 초기화하여 처음부터 학습

PPO의 정책 네트워크(actor)와 가치 함수(critic)는
SB3 내부에서 자동으로 생성됩니다.

<br>

### 2.4 PPO 학습 루프

PPO 에이전트가 생성되면,
`learn()` 함수를 통해 학습을 수행합니다.

`learn()` 내부에서는 다음 과정이 반복됩니다.

1. 현재 정책으로 병렬 환경과 상호작용하며 rollout 수집
2. GAE를 이용한 advantage 계산
3. 정책 및 가치 함수 업데이트
4. 학습 로그 기록 및 콜백 실행

<div align="center">
  <img src="https://drive.google.com/uc?id=1jm4dvSTDlkRLh4HcoiYA-y8PsIiRuhui">
</div>


---
## 3. Training and Finetuning

Training quadruped locomotion policies from scratch can be time-consuming.
Therefore, this tutorial uses a **pretrained policy** as a starting point and
performs additional training (finetuning) in the target environment.

- If `USE_PRETRAINED = True`, a pretrained checkpoint is loaded and training continues.
- If `USE_PRETRAINED = False`, the policy is initialized from scratch.

This approach significantly reduces training time while still allowing
reward design and environment settings to influence the learned behavior.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/RL_DEMO/logs

The tensorboard extension is already loaded. To reload it, use:

%reload_ext tensorboard

Reusing TensorBoard on port 6006 (pid 1727), started 0:15:21 ago. (Use '!kill 1727' to kill it.)

<IPython.core.display.Javascript object>

In [ ]:
importlib.reload(go1_env)

USE_PRETRAINED = True
PRETRAINED_MODEL_PATH = "/content/RL_DEMO/models/pretrained2/final_model.zip"

# Train
MODEL_DIR = "models"
LOG_DIR = "logs"

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

vec_env = make_vec_env(
    go1_env.Go1MujocoEnv,
    env_kwargs={"prj_path": "/content/RL_DEMO"},
    n_envs=policy_cfg["n_envs"],
    seed=policy_cfg["seed"],
    vec_env_cls=SubprocVecEnv,
)

train_time = time.strftime("%Y-%m-%d_%H-%M-%S")
run_name = f"{train_time}"

model_path = f"{MODEL_DIR}/{run_name}"
print(
    f"Training on {policy_cfg['n_envs']} parallel training environments and saving models to '{model_path}'"
)

checkpoint_callback = CheckpointCallback(
    save_freq=policy_cfg["policy"]["n_steps"] * policy_cfg["log"]["interval"],  # e.g. 100_000
    save_path=model_path,  # directory
    name_prefix="model",  # checkpoint_model_100000_steps.zip
    save_replay_buffer=False,
    save_vecnormalize=False,
)

eval_callback = EvalCallback(
    vec_env,
    best_model_save_path=model_path,
    log_path=LOG_DIR,
    eval_freq=policy_cfg["eval_freq"],
    n_eval_episodes=5,
    deterministic=True,
    render=False,
)

reward_logging_callback = RewardLoggingCallback()

callbacks = CallbackList([
    eval_callback,
    checkpoint_callback,
    reward_logging_callback,
])

if USE_PRETRAINED:
    print(f"Using Pretrained model from {PRETRAINED_MODEL_PATH}")
    model = PPO.load(path=PRETRAINED_MODEL_PATH, env=vec_env)
    model.tensorboard_log = LOG_DIR
else:
    print("Training from Network model from scratch")
    model = PPO("MlpPolicy", vec_env, verbose=1, tensorboard_log=LOG_DIR)

model.learn(
    total_timesteps=policy_cfg["total_timestep"],
    reset_num_timesteps=True,
    progress_bar=True,
    tb_log_name=run_name,
    callback=callbacks,
)
# Save final model
model.save(f"{model_path}/final_model")

vec_env.close()

del model
del eval_callback
del vec_env

gc.collect()

Training on 12 parallel training environments and saving models to 'models/2026-02-07_14-57-50'
Using Pretrained model from /content/RL_DEMO/models/pretrained2/final_model.zip


/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


Logging to logs/2026-02-07_14-57-50_1


Output()

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

...

Episode length: 750.00 +/- 0.00

----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 750        |
|    mean_reward          | 1.19e+03   |
| time/                   |            |
|    total_timesteps      | 120000     |
| train/                  |            |
|    approx_kl            | 0.09673851 |
|    clip_fraction        | 0.516      |
|    clip_range           | 0.2        |
|    entropy_loss         | 19.8       |
|    explained_variance   | 0.4071598  |
|    learning_rate        | 0.0002     |
|    loss                 | 9.21       |
|    n_updates            | 5450       |
|    policy_gradient_loss | 0.00585    |
|    std                  | 0.0515     |
|    value_loss           | 107        |
----------------------------------------


New best mean reward!

----------------------------------------
| cost/                  |             |
|    action_norm         | 0.010422045 |
|    action_rate         | 0.0054      |
|    foot_clearance      | 0.28        |
|    gait_enforcement    | 0.3         |
|    joint_acc           | 0.0083      |
|    joint_lim           | 0           |
|    joint_pos_deviation | 0.0679      |
|    termination         | 0           |
|    torque              | 0.218       |
|    vertical_vel        | 0.00708     |
|    xy_angular_vel      | 0.0177      |
| reward/                |             |
|    ang_vel             | 0.986       |
|    base_height_reward  | 0           |
|    feet_air_time       | 0           |
|    healthy             | 0           |
|    lin_vel             | 1.47        |
|    total               | 1.54        |
|    total_costs         | 0.915       |
|    total_rewards       | 2.45        |
| rollout/               |             |
|    ep_len_mean         | 743         |
|    ep_rew_mean

KeyboardInterrupt

## 4. Policy Evaluation

After training, the learned policy is evaluated in a single environment.
The robot’s behavior is rendered and saved as a video for qualitative analysis.

- Control frequency: 50 Hz
- Video frame rate: 25 FPS
- Frames are sampled periodically and saved as an MP4 file

The following cell generates and displays a rollout video.


In [ ]:
# Test
importlib.reload(go1_env)
model_path = "/content/RL_DEMO/models/pretrained2/final_model"

WIDTH, HEIGHT = 544, 368

env = go1_env.Go1MujocoEnv(
    prj_path="/content/RL_DEMO",
    render_mode="rgb_array",
    camera_name="tracking",
    width=WIDTH,
    height=HEIGHT,
)
inter_frame_sleep = 0.0

model = PPO.load(path=model_path, env=env, verbose=1)

video_path = "/content/rollout.mp4"

obs, _ = env.reset()
max_time_step_s = policy_cfg["test"]["max_time_step_s"]
ep_len = 0
video_fps = 25

# Ctrl Hz: 50
render_interval = 50 // video_fps
max_steps = int(max_time_step_s * 50)

writer = imageio.get_writer(
    video_path,
    fps=video_fps,
    codec="libx264",
    quality=8,
    pixelformat="yuv420p",
)

frames = []

while ep_len < max_steps:
  with torch.no_grad():
    action, _ = model.predict(obs, deterministic=True)
  obs, reward, terminated, truncated, info = env.step(action)

  if ep_len % render_interval == 0:
    frame = env.render()
    frames.append(frame)

  ep_len += 1

imageio.mimwrite(
    video_path,
    frames,
    fps=video_fps,
    codec="libx264",
    quality=8,
    pixelformat="yuv420p",
)

env.close()

print("Saved video to:", video_path)

display(
    Video(
        video_path,
        embed=True,
        html_attributes="controls autoplay loop"
    )
)

/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


## 5. Reward Ablation Study

Finally, we examine how reward design affects locomotion behavior.

By loading checkpoints trained with different reward configurations
(e.g., removing smoothness or energy penalties), we can directly observe
how each reward term influences stability, motion smoothness, and failure modes.

In the following cells, we compare rollout videos from different checkpoints
under identical evaluation conditions.
